In [2]:
import numpy as np

from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score

import torch.utils.data as Data
import torch

In [3]:
data_train = np.load('Dataset/data_train.npy')
X = data_train[:,:3]
Y = data_train[:,-1]
print(X.shape)
print(Y.shape)

(2400000, 3)
(2400000,)


In [4]:
standarder = StandardScaler()
X_train , X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.2, random_state=0)
standarder.fit(X_train)
X_train = standarder.transform(X_train)
X_test = standarder.transform(X_test)
X_train = torch.from_numpy(X_train)
#X_test = torch.from_numpy(X_test)
Y_train = torch.from_numpy(Y_train)
#Y_test = torch.from_numpy(Y_test)

In [5]:
data_train = Data.TensorDataset(X_train, Y_train)
#data_test = Data.TensorDataset(X_test, Y_test)

In [10]:
BATCH_SIZE = 65536
loder_train = Data.DataLoader(data_train, batch_size= BATCH_SIZE, shuffle= True)
#loder_test = Data.DataLoader(data_test, batch_size= BATCH_SIZE, shuffle= False)

In [11]:
svm = joblib.load('model/svm.joblib')
sgd = joblib.load('model/sgd.joblib')
randonforest = joblib.load('model/randomforest.joblib')
model = VotingClassifier(estimators=[('svm',svm), ('sgd',sgd), ('randonforest', randonforest)], voting= 'soft', n_jobs= -1)

In [12]:
for step,(img, label) in enumerate(loder_train):

    model.fit(img.numpy(), label.numpy())
    print(model.score(X_test,Y_test))
    
    break

0.8596541666666667


In [13]:
joblib.dump(model, 'model/ensemble.joblib')

['model/ensemble.joblib']